# Housing Affordability Project

### Leading questions

- What metros have the highest incidence of rent burden? Severe rent burden?
- How has the incidence of rent burden and severe rent burden changed over time?
- Do patterns emerge between a metro's incidence of rent burden and other metro-wide factors?

Install required packages for SQL functionality and Census Bureau API calls

In [ ]:
%pip install pandas sqlalchemy jupysql census censusdata requests

Import packages <br>
*NOTE*: need to hide API key before committing to GitHub

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from census import Census
from config import CENSUS_API_KEY

Use JupySQL and SQLAlchemy to create and establish connection to a SQLite database

In [2]:
%load_ext sql
engine = create_engine('sqlite:///housing.db')
%sql sqlite:///housing.db

Connecting to 'sqlite:///housing.db'

# ZORI import

Import Zillow/ZORI data from CSV, save as a pandas dataframe <br>
*NOTE:* ZORI data: smoothed, seasonally adjusted, all homes

In [ ]:
zori_df = pd.read_csv('../zori_msa.csv')
zori_df.head()

In [ ]:
zori_df.to_sql('zori', con=engine, index=False, if_exists='replace')

In [ ]:
%%sql
SELECT RegionName
FROM zori
WHERE RegionName LIKE '%Seattle, WA%'
OR RegionName LIKE '%San Francisco, CA%'
OR RegionName LIKE '%New York, NY%'
OR RegionName LIKE '%Atlanta, GA%'
OR RegionName LIKE '%Phoenix, AZ%'
OR RegionName LIKE '%Austin, TX%'
OR RegionName LIKE '%Minneapolis, MN%'
OR RegionName LIKE '%Detroit, MI%'
OR RegionName LIKE '%St. Louis, MO%'

In [ ]:
%sqlcmd tables

In [ ]:
%%sql
DROP TABLE IF EXISTS rent_to_income_23

# Census Import

Set up parameters for Census API pull

In [3]:
c = Census(CENSUS_API_KEY)
variables = (
    "NAME",
    "B25070_001E",  # Total
    "B25070_002E",  # Less than 10.0
    "B25070_003E",  # 10.0 to 14.9
    "B25070_004E",  # 15.0 to 19.9
    "B25070_005E",  # 20.0 to 24.9
    "B25070_006E",  # 25.0 to 29.9
    "B25070_007E",  # 30.0 to 34.9
    "B25070_008E",  # 35.0 to 39.9
    "B25070_009E",  # 40.0 to 49.9
    "B25070_010E",  # 50.0 or more
    "B25070_011E"   # Not computed
)

Pull data <br>
*rent_to_income_raw* = list type

In [4]:
rent_to_income_raw = c.acs1.get(
    variables,
    {'for': 'metropolitan statistical area/micropolitan statistical area:*'},
    year=2023
)

# Data Cleaning Process / Workflow

### Step 0: Save pulled data as a pandas dataframe

In [22]:
rent_to_income_df = pd.DataFrame(rent_to_income_raw)

### Step 1: Conduct initial inspection of data

In [23]:
rent_to_income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 13 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   NAME                                                         530 non-null    object 
 1   B25070_001E                                                  529 non-null    float64
 2   B25070_002E                                                  529 non-null    float64
 3   B25070_003E                                                  529 non-null    float64
 4   B25070_004E                                                  529 non-null    float64
 5   B25070_005E                                                  529 non-null    float64
 6   B25070_006E                                                  529 non-null    float64
 7   B25070_007E                                                  529 non-null    flo

Why do columns 0 and 12 have a different non-null count?

In [26]:
rent_to_income_df.isnull().sum()

NAME                                                           0
B25070_001E                                                    1
B25070_002E                                                    1
B25070_003E                                                    1
B25070_004E                                                    1
B25070_005E                                                    1
B25070_006E                                                    1
B25070_007E                                                    1
B25070_008E                                                    1
B25070_009E                                                    1
B25070_010E                                                    1
B25070_011E                                                    1
metropolitan statistical area/micropolitan statistical area    0
dtype: int64

In [27]:
rent_to_income_df[rent_to_income_df["B25070_001E"].isnull()]

,NAME,B25070_001E,B25070_002E,B25070_003E,B25070_004E,B25070_005E,B25070_006E,B25070_007E,B25070_008E,B25070_009E,B25070_010E,B25070_011E,metropolitan statistical area/micropolitan statistical area
323,"Murrells Inlet, SC Micro Area",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34680


This Micro Area is missing data for this year. Not too relevant since it's not a part of our analysis. No harm in removing it

In [ ]:
rent_to_income_df = rent_to_income_df.dropna()
rent_to_income_df.info()

### Step 2: Rename columns

In [ ]:
rent_to_income_df.rename(columns={
    "NAME": "msa_name",
    "B25070_001E": "hh_total",
    "B25070_002E": "hh_below_10",
    "B25070_003E": "hh_10_to_15",
    "B25070_004E": "hh_15_to_20",
    "B25070_005E": "hh_20_to_25",
    "B25070_006E": "hh_25_to_30",
    "B25070_007E": "hh_30_to_35",
    "B25070_008E": "hh_35_to_40",
    "B25070_009E": "hh_40_to_50",
    "B25070_010E": "hh_above_50",
    "B25070_011E": "not_computed",
    "metropolitan statistical area/micropolitan statistical area": "cbsa_code"
}, inplace=True)

In [ ]:
rent_to_income_df.head()

### Step 3: Add and drop columns

Adding a year column, which will be useful later after importing other years' data.

In [ ]:
rent_to_income_df["year"] = 2023

Create a derived total column that contains the total amount of computed households (hh_total less not_computed) <br>
This will be the only "total" worth using for analysis

In [ ]:
rent_to_income_df["hh_total_computed"] = rent_to_income_df["hh_total"] - rent_to_income_df["not_computed"]

Before deleting hh_total and not_computed, verify that my computed column is the sum of sub-columns

NOTE:
- dataframe axis=0 or axis=1?
- dataframe.all() returns true unless at least one element is false or equivalent

In [ ]:
check = rent_to_income_df["hh_total_computed"] == rent_to_income_df[["hh_below_10", "hh_10_to_15", "hh_15_to_20", 
"hh_20_to_25", "hh_25_to_30", "hh_30_to_35", "hh_35_to_40", "hh_40_to_50", "hh_above_50"]].sum(axis=1)

check.all()

In [ ]:
rent_to_income_df = rent_to_income_df.drop(columns=["hh_total", "not_computed"], inplace=False)
rent_to_income_df.head()

### Step 4: Filter/delete rows

For this project, I am looking at data for 9 MSAs

Since exact MSA names can change year over year, use CBSA codes, which are stable over time, for best reproducibility.
- seattle: 42660
- SF: 41860
- NYC: 35620
- atlanta: 12060
- phoenix: 38060
- austin: 12420
- minneapolis: 33460
- detroit: 19820
- st louis: 41180

In [ ]:
# rent_to_income_df[rent_to_income_df["msa_name"].str.contains("st. louis", case=False, na=False)]

target_cbsa_codes = ["42660", "41860", "35620", "12060", "38060",
                     "12420", "33460", "19820", "41180"]

Boolean indexing, reassign df of 9 relevant MSAs to itself

In [ ]:
rent_to_income_df = rent_to_income_df[rent_to_income_df["cbsa_code"].isin(target_cbsa_codes)]

In [ ]:
rent_to_income_df.info()
rent_to_income_df

### Step 5: Deal with missing data

Not a problem for 2023

### Step 6: Reorder columns

In [ ]:
desired_order = ['cbsa_code', 'msa_name', 'year', 'hh_total_computed', 'hh_below_10',
                 'hh_10_to_15', 'hh_15_to_20', 'hh_20_to_25', 'hh_25_to_30', 'hh_30_to_35',
                 'hh_35_to_40', 'hh_40_to_50', 'hh_above_50']

rent_to_income_df = rent_to_income_df[desired_order]

In [ ]:
rent_to_income_df

### Step X: Repeat for other years

In [ ]:
df_merged = pd.DataFrame()

df_merged = pd.concat([df_merged, rent_to_income_df], ignore_index=True)

In [ ]:
df_merged.info()

Set up a big loop, for each year (2016 to 2023):
1. extract data from census API
2. store data in dataframe
3. rename columns
4. add "year" and "total_computed" columns
5. drop "hh_total" and "not_computed" columns
6. filter to 9 msas
7. reorder columns
8. add dataframe to df_merged

note:
- check that "total_computed" is a correct sum
- expect missing values in 2020 for relevant MSAs

### Define variables needed for loop

In [20]:
c = Census(CENSUS_API_KEY)

census_variables = (
    "NAME",
    "B25070_001E",  # Total
    "B25070_002E",  # Less than 10.0
    "B25070_003E",  # 10.0 to 14.9
    "B25070_004E",  # 15.0 to 19.9
    "B25070_005E",  # 20.0 to 24.9
    "B25070_006E",  # 25.0 to 29.9
    "B25070_007E",  # 30.0 to 34.9
    "B25070_008E",  # 35.0 to 39.9
    "B25070_009E",  # 40.0 to 49.9
    "B25070_010E",  # 50.0 or more
    "B25070_011E"   # Not computed
)

target_msa_codes = ["42660", "41860", "35620", "12060", "38060",
                     "12420", "33460", "19820", "41180"]

# column_order = ['msa_code', 'msa_name', 'year', 'hh_total_computed', 'hh_below_10',
#                 'hh_10_to_15', 'hh_15_to_20', 'hh_20_to_25', 'hh_25_to_30', 'hh_30_to_35',
#                 'hh_35_to_40', 'hh_40_to_50', 'hh_above_50']

In [21]:
merged_df = pd.DataFrame()

### Loop

In [22]:
for year_loop in [2016, 2017, 2018, 2019, 2021, 2022, 2023]:
    print(year_loop)
    
    # extract data from census API
    b25070_raw = c.acs1.get(
        census_variables,
        {'for': 'metropolitan statistical area/micropolitan statistical area:*'},
        year=year_loop
    )    

    
    # store data in dataframe
    single_year_df = pd.DataFrame(b25070_raw)

    
    # rename columns
    single_year_df = single_year_df.rename(columns={
        "NAME": "msa_name",
        "B25070_001E": "total_households_raw",
        "B25070_002E": "less_than_100",
        "B25070_003E": "100_to_149",
        "B25070_004E": "150_to_199",
        "B25070_005E": "200_to_249",
        "B25070_006E": "250_to_299",
        "B25070_007E": "300_to_349",
        "B25070_008E": "350_to_399",
        "B25070_009E": "400_to_499",
        "B25070_010E": "500_or_more",
        "B25070_011E": "households_not_computed",
        "metropolitan statistical area/micropolitan statistical area": "msa_code"
    }, inplace=False)   


    
    # add "year" and "total_computed" columns
    single_year_df["year"] = year_loop
    
    single_year_df["total_households"] = single_year_df["total_households_raw"] - single_year_df["households_not_computed"]

    
    # drop "hh_total" and "not_computed" columns
    single_year_df = single_year_df.drop(columns=["total_households_raw", "households_not_computed"], inplace=False)

    
    # filter to 9 msas
    single_year_df = single_year_df[single_year_df["msa_code"].isin(target_cbsa_codes)]

    
    # reorder columns
    # single_year_df = single_year_df[column_order]

    
    # add dataframe to df_merged
    merged_df = pd.concat([merged_df, single_year_df], ignore_index=True)

print("done")

2016
2017
2018
2019
2021
2022
2023
done


In [23]:
merged_df.head()

,msa_name,less_than_100,100_to_149,150_to_199,200_to_249,250_to_299,300_to_349,350_to_399,400_to_499,500_or_more,msa_code,year,total_households
0,"Detroit-Warren-Dearborn, MI Metro Area",20173.0,48142.0,66324.0,65809.0,51797.0,43616.0,31464.0,46630.0,132940.0,19820,2016,506895.0
1,"St. Louis, MO-IL Metro Area",15962.0,32938.0,46123.0,44480.0,39146.0,25854.0,21895.0,28795.0,74818.0,41180,2016,330011.0
2,"San Francisco-Oakland-Hayward, CA Metro Area",39884.0,72610.0,102225.0,100233.0,85421.0,64276.0,47431.0,65088.0,174925.0,41860,2016,752093.0
3,"Seattle-Tacoma-Bellevue, WA Metro Area",19758.0,48147.0,77595.0,84380.0,73227.0,54981.0,36522.0,50927.0,123698.0,42660,2016,569235.0
4,"Minneapolis-St. Paul-Bloomington, MN-WI Metro ...",14308.0,37762.0,55986.0,61677.0,49631.0,38184.0,25435.0,33641.0,87397.0,33460,2016,404021.0


### Check for missing data and validity of computed column

In [24]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   msa_name          63 non-null     object 
 1   less_than_100     63 non-null     float64
 2   100_to_149        63 non-null     float64
 3   150_to_199        63 non-null     float64
 4   200_to_249        63 non-null     float64
 5   250_to_299        63 non-null     float64
 6   300_to_349        63 non-null     float64
 7   350_to_399        63 non-null     float64
 8   400_to_499        63 non-null     float64
 9   500_or_more       63 non-null     float64
 10  msa_code          63 non-null     object 
 11  year              63 non-null     int64  
 12  total_households  63 non-null     float64
dtypes: float64(10), int64(1), object(2)
memory usage: 6.5+ KB


In [25]:
check = merged_df["total_households"] == merged_df[["less_than_100", "100_to_149", "150_to_199", 
"200_to_249", "250_to_299", "300_to_349", "350_to_399", "400_to_499", "500_or_more"]].sum(axis=1)

check.all()

np.True_

### Wide -> Long conversion using pandas.melt

id_vars = columns to stay fixed and used as identifier variables <br>
value_vars = columns to unpivot into rows <br>
var_name = name of new column that will store "previous column names" <br>
value_name = name of new column that will store "previous values" <br>

In [29]:
merged_df.columns

Index(['msa_name', 'less_than_100', '100_to_149', '150_to_199', '200_to_249',
       '250_to_299', '300_to_349', '350_to_399', '400_to_499', '500_or_more',
       'msa_code', 'year', 'total_households'],
      dtype='object')

In [30]:
id_columns = merged_df.columns[[0, 10, 11, 12]]

In [31]:
value_columns = merged_df.columns[1:10]

In [32]:
id_columns, value_columns

(Index(['msa_name', 'msa_code', 'year', 'total_households'], dtype='object'),
 Index(['less_than_100', '100_to_149', '150_to_199', '200_to_249', '250_to_299',
        '300_to_349', '350_to_399', '400_to_499', '500_or_more'],
       dtype='object'))

In [33]:
merged_df = pd.melt(
    merged_df,
    id_vars = id_columns,
    value_vars = value_columns,
    var_name = "rent_burden_category",
    value_name = "households"
)

In [36]:
column_order = [
    "msa_code", "msa_name", "year", "rent_burden_category", "households", "total_households"
]
merged_df = merged_df[column_order]
merged_df.head()

,msa_code,msa_name,year,rent_burden_category,households,total_households
0,19820,"Detroit-Warren-Dearborn, MI Metro Area",2016,less_than_100,20173.0,506895.0
1,41180,"St. Louis, MO-IL Metro Area",2016,less_than_100,15962.0,330011.0
2,41860,"San Francisco-Oakland-Hayward, CA Metro Area",2016,less_than_100,39884.0,752093.0
3,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2016,less_than_100,19758.0,569235.0
4,33460,"Minneapolis-St. Paul-Bloomington, MN-WI Metro ...",2016,less_than_100,14308.0,404021.0


## next to do:

- standardize names for var_name, value_name, and values of var_name (the original column names)
- move this df to SQL (?)
- what to do with 2020 data?
- make buckets of rent burden (<30%, 30-40%, 40-50%, >50%)

In [38]:
merged_df.loc[
    (merged_df["year"] == 2023) & (merged_df["msa_name"].str.contains("seattle", case=False)),
    :
]

,msa_code,msa_name,year,rent_burden_category,households,total_households
62,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2023,less_than_100,26376.0,634047.0
125,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2023,100_to_149,58134.0,634047.0
188,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2023,150_to_199,84136.0,634047.0
251,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2023,200_to_249,81065.0,634047.0
314,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2023,250_to_299,72359.0,634047.0
377,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2023,300_to_349,59258.0,634047.0
440,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2023,350_to_399,48272.0,634047.0
503,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2023,400_to_499,55604.0,634047.0
566,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2023,500_or_more,148843.0,634047.0


In [41]:
# create buckets

bucket_map = {
    "less_than_100": "Not rent burdened",
    "100_to_149": "Not rent burdened",
    "150_to_199": "Not rent burdened",
    "200_to_249": "Not rent burdened",
    "250_to_299": "Not rent burdened",
    "300_to_349": "Moderately rent burdened",
    "350_to_399": "Moderately rent burdened",
    "400_to_499": "Moderately rent burdened",
    "500_or_more": "Severely rent burdened"
}

merged_df["rent_burden_group"] = merged_df["rent_burden_category"].map(bucket_map)

In [42]:
merged_df.head()

,msa_code,msa_name,year,rent_burden_category,households,total_households,rent_burden_group
0,19820,"Detroit-Warren-Dearborn, MI Metro Area",2016,less_than_100,20173.0,506895.0,Not rent burdened
1,41180,"St. Louis, MO-IL Metro Area",2016,less_than_100,15962.0,330011.0,Not rent burdened
2,41860,"San Francisco-Oakland-Hayward, CA Metro Area",2016,less_than_100,39884.0,752093.0,Not rent burdened
3,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2016,less_than_100,19758.0,569235.0,Not rent burdened
4,33460,"Minneapolis-St. Paul-Bloomington, MN-WI Metro ...",2016,less_than_100,14308.0,404021.0,Not rent burdened


In [17]:
df.loc[
    (df["msa_name"].str.contains("seattle", case=False)) & (df["year"] == 2016),
    :
]

,cbsa_code,msa_name,year,hh_total_computed,rent_burden_category,hh_count,rent_burden_group
3,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2016,569235.0,hh_below_10,19758.0,not rent burdened
66,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2016,569235.0,hh_10_to_15,48147.0,not rent burdened
129,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2016,569235.0,hh_15_to_20,77595.0,not rent burdened
192,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2016,569235.0,hh_20_to_25,84380.0,not rent burdened
255,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2016,569235.0,hh_25_to_30,73227.0,not rent burdened
318,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2016,569235.0,hh_30_to_35,54981.0,rent burdened
381,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2016,569235.0,hh_35_to_40,36522.0,rent burdened
444,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2016,569235.0,hh_40_to_50,50927.0,rent burdened
507,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2016,569235.0,hh_above_50,123698.0,severely rent burdened


#### Using a SQL-like group-by aggregation, create a new DataFrame that only has the rent_burden_group information
#### This is more EDA-ready, as it is less granular (more coarse)

In [13]:
df_groups = df.groupby(['cbsa_code', 'msa_name', 'year', "hh_total_computed", 'rent_burden_group'], as_index=False)['hh_count'].sum()

In [14]:
df_groups["hh_pct"] = df_groups["hh_count"] / df_groups["hh_total_computed"]

In [15]:
df_groups.loc[
    df_groups["msa_name"].str.contains("seattle", case=False),
    :
]

,cbsa_code,msa_name,year,hh_total_computed,rent_burden_group,hh_count,hh_pct
168,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2016,569235.0,not rent burdened,303107.0,0.532481
169,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2016,569235.0,rent burdened,142430.0,0.250213
170,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2016,569235.0,severely rent burdened,123698.0,0.217306
171,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2017,569710.0,not rent burdened,297943.0,0.522973
172,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2017,569710.0,rent burdened,144835.0,0.254226
173,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2017,569710.0,severely rent burdened,126932.0,0.222801
174,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2018,591088.0,not rent burdened,311341.0,0.526725
175,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2018,591088.0,rent burdened,148271.0,0.250844
176,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2018,591088.0,severely rent burdened,131476.0,0.222431
177,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2019,596076.0,not rent burdened,314136.0,0.527007


In [16]:
df_groups.loc[
    (df_groups["year"] == 2023) &
    (df_groups["rent_burden_group"] == "not rent burdened")
]

,cbsa_code,msa_name,year,hh_total_computed,rent_burden_group,hh_count,hh_pct
18,12060,"Atlanta-Sandy Springs-Roswell, GA Metro Area",2023,733261.0,not rent burdened,316727.0,0.431943
39,12420,"Austin-Round Rock-San Marcos, TX Metro Area",2023,415896.0,not rent burdened,196913.0,0.473467
60,19820,"Detroit-Warren-Dearborn, MI Metro Area",2023,455084.0,not rent burdened,217158.0,0.477182
81,33460,"Minneapolis-St. Paul-Bloomington, MN-WI Metro ...",2023,428780.0,not rent burdened,224122.0,0.522697
84,35620,"New York-Newark-Jersey City, NY-NJ Metro Area",2023,3482685.0,not rent burdened,1679361.0,0.482203
114,38060,"Phoenix-Mesa-Chandler, AZ Metro Area",2023,600238.0,not rent burdened,271128.0,0.451701
144,41180,"St. Louis, MO-IL Metro Area",2023,318502.0,not rent burdened,169502.0,0.532185
156,41860,"San Francisco-Oakland-Fremont, CA Metro Area",2023,750585.0,not rent burdened,381777.0,0.508639
186,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",2023,634047.0,not rent burdened,322070.0,0.507959


### NEXT STEPS: the above table is the format I want, but clean it up (and clean up the code). clean up column names and order. then maybe I can start doing some basic visualizations with rent burden data. can ignore zillow for now.

In [ ]:
rent_to_income_df.to_sql('rent_to_income_2023', con=engine, index=False, if_exists='replace')

In [ ]:
%%sql
SELECT msa_code, msa, total_households, less_10, at_least_50, 2023 AS year
FROM rent_to_income_23
WHERE msa LIKE "%seattle%"
    OR msa LIKE "%san francisco%"
    OR msa LIKE "%new york%"
    OR msa LIKE "%atlanta%"
    OR msa LIKE "%phoenix%"
    OR msa LIKE "%austin%"
    OR msa LIKE "%minneapolis%"
    OR msa LIKE "%detroit%"
    OR msa LIKE "%st. louis%"